In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

BASE_DIR = r"D:\Data\SCADA\Wind_Turbine"

In [2]:
def get_farm_scada_chunked(farm='A', asset_ids=None, chunksize=50000):
    """
    Process farm data in chunks - for when dataset won't fit in RAM.
    
    asset_ids: Process only specific assets (e.g., [0, 10, 21])
    chunksize: Rows per chunk
    """
    farm_dir = os.path.join(BASE_DIR, 'Wind Farm '+farm)
    farm_dataset_dir = os.path.join(farm_dir, 'datasets')
    all_files = glob.glob(os.path.join(farm_dataset_dir, '*.csv'))
    parquet_path = os.path.join(farm_dir, f'farm_{farm}_optimized.parquet')
    dtype_dict = {
        'asset_id': 'int16',
        'status_type_id': 'int8',
    }
    # Check if already processed
    if os.path.exists(parquet_path):
        print(f"Loading pre-processed Farm {farm}...")
        return pd.read_parquet(parquet_path)
        
    all_data = []
    
    for f in all_files:
        # Read in chunks
        for chunk in pd.read_csv(f, sep=";", dtype=dtype_dict, chunksize=chunksize):
            # Filter to specific assets if provided
            if asset_ids is not None:
                chunk = chunk[chunk['asset_id'].isin(asset_ids)]
            
            # Optimize dtypes
            float_cols = chunk.select_dtypes(include=['float64']).columns
            chunk[float_cols] = chunk[float_cols].astype('float32')
            
            # Clean
            chunk["time_stamp"] = pd.to_datetime(chunk["time_stamp"])
            chunk = chunk.drop(['train_test', 'id'], axis=1, errors='ignore')
            chunk = chunk.dropna(subset=["asset_id", "time_stamp"])
            
            all_data.append(chunk)
    
    scada_data = pd.concat(all_data, ignore_index=True)
    scada_data = scada_data.drop_duplicates(subset=['asset_id', 'time_stamp'], keep='first')
    scada_data = scada_data.sort_values(["asset_id", "time_stamp"]).reset_index(drop=True)

    scada_data.to_parquet(parquet_path, compression='snappy', index=False)
    print(f"Saved to {parquet_path}")
    
    return scada_data

In [3]:
def get_event_info(farm='A'):
    farm_dir = os.path.join(BASE_DIR, 'Wind Farm '+farm)
    event_info = pd.read_csv(farm_dir + '\\event_info.csv', sep=';')
    event_info = event_info.rename(columns={'asset':'asset_id'})
    # Drop events that aren't anomalies
    event_info = event_info[event_info['event_label'] == 'anomaly']
    # Clean Up
    event_info["event_start"] = pd.to_datetime(event_info["event_start"])
    event_info["event_end"] = pd.to_datetime(event_info["event_end"])
    event_info["asset_id"] = pd.to_numeric(event_info["asset_id"], errors="coerce").astype("Int16")
    # Drop rows with missing critical info
    event_info = event_info.dropna(subset=["asset_id", "event_start", 'event_end'])
    # Sort for asset_id
    event_info = event_info.sort_values(["asset_id"]).reset_index(drop=True)

    return event_info

In [10]:
def build_unified_dataset(scada_dict, event_info_dict, sensor_mapping, 
                         power_sensors, power_threshold=0.1, 
                         window_hours=24, buffer_days=30, random_seed=42):
    
    all_windows = []
    
    for farm in ['A', 'B', 'C']:
        print(f"\nProcessing Farm {farm}...")
        
        scada = scada_dict[farm]
        events = event_info_dict[farm]
        power_col = power_sensors[farm]
        
        failure_windows = extract_failure_windows_unified_fixed(
            scada, events, farm, sensor_mapping, 
            power_col, power_threshold, window_hours
        )
        
        print(f"  Failure windows: {len(failure_windows)}")
        
        normal_windows = extract_normal_windows_unified(
            scada, events, farm, sensor_mapping,
            power_col, power_threshold, window_hours, buffer_days,random_seed=random_seed
        )
        
        print(f"  Normal windows: {len(normal_windows)}")
        
        # Combine
        all_windows.extend(failure_windows)
        all_windows.extend(normal_windows)
    
    dataset = pd.DataFrame(all_windows)
    
    print(f"\n{'='*60}")
    print(f"UNIFIED DATASET CREATED")
    print(f"{'='*60}")
    print(f"Total windows: {len(dataset)}")
    print(f"Failure windows: {(dataset['label']==1).sum()}")
    print(f"Normal windows: {(dataset['label']==0).sum()}")
    print(f"Features: {len([c for c in dataset.columns if c not in ['label', 'event_id', 'asset_id', 'farm', 'failure_type']])}")
    print(f"\nWindows per farm:")
    print(dataset['farm'].value_counts())
    
    return dataset
    print('Unified DB built')

In [12]:
def extract_failure_windows_unified_fixed(scada, events, farm, sensor_mapping,
                                         power_col, power_threshold, window_hours):    
    windows = []
    
    for _, failure in events[events['event_label'] == 'anomaly'].iterrows():
        
        asset_id = failure['asset_id']
        event_id = failure['event_id']
        failure_start = failure['event_start']
        
        # Find last production time
        production_data = scada[
            (scada['asset_id'] == asset_id) &
            (scada['time_stamp'] < failure_start) &
            (scada['status_type_id'] == 0) &
            (scada[power_col] > power_threshold)
        ].sort_values('time_stamp')
        
        if len(production_data) == 0:
            continue
        
        last_production = production_data.iloc[-1]['time_stamp']
        window_start = last_production - pd.Timedelta(hours=window_hours)
        
        window_data = scada[
            (scada['asset_id'] == asset_id) &
            (scada['time_stamp'] >= window_start) &
            (scada['time_stamp'] <= last_production)
        ]
        
        if len(window_data) < 100:
            continue
        
        features = extract_unified_window_features_fixed(
            window_data, farm, sensor_mapping,
            statistics=['mean', 'std', 'trend']
        )
        
        features['label'] = 1
        features['event_id'] = event_id
        features['asset_id'] = asset_id
        features['farm'] = farm
        features['failure_type'] = failure.get('event_description', 'Unknown')
        
        windows.append(features)
    
    return windows

In [20]:
def extract_normal_windows_unified(scada, events, farm, sensor_mapping,
                                   power_col, power_threshold, window_hours,
                                   buffer_days, windows_per_asset=20, random_seed=42):
    np.random.seed(random_seed)
    windows = []
    
    assets = scada['asset_id'].unique()
    
    for asset_id in assets:
        
        asset_failures = events[
            (events['asset_id'] == asset_id) &
            (events['event_label'] == 'anomaly')
        ]   
        exclusion_zones = []
        for _, failure in asset_failures.iterrows():
            start_exclude = failure['event_start'] - pd.Timedelta(days=buffer_days)
            end_exclude = failure['event_end'] + pd.Timedelta(days=buffer_days)
            exclusion_zones.append((start_exclude, end_exclude))
        
        asset_data = scada[
            (scada['asset_id'] == asset_id) &
            (scada['status_type_id'] == 0) &
            (scada[power_col] > power_threshold)
        ].copy()
        
        mask = pd.Series(True, index=asset_data.index)
        for start_ex, end_ex in exclusion_zones:
            mask &= ~((asset_data['time_stamp'] >= start_ex) & 
                      (asset_data['time_stamp'] <= end_ex))
        
        normal_data = asset_data[mask].sort_values('time_stamp')
        
        if len(normal_data) < 144 * windows_per_asset:
            continue
        
        sampled = 0
        max_attempts = windows_per_asset * 3
        attempts = 0
        
        while sampled < windows_per_asset and attempts < max_attempts:
            attempts += 1
            
            max_start_idx = len(normal_data) - 145
            if max_start_idx < 0:
                break
            
            start_idx = np.random.randint(0, max_start_idx)
            window_candidate = normal_data.iloc[start_idx:start_idx + 145]
            
            time_diffs = window_candidate['time_stamp'].diff()
            if time_diffs.max() > pd.Timedelta(minutes=30):
                continue
            
            features = extract_unified_window_features_fixed(
                window_data=window_candidate,  
                farm=farm,
                sensor_mapping=sensor_mapping,
                statistics=['mean', 'std', 'trend']
            )
            
            features['label'] = 0
            features['event_id'] = None
            features['asset_id'] = asset_id
            features['farm'] = farm
            features['failure_type'] = 'normal'
            
            windows.append(features)
            sampled += 1
    
    return windows

In [14]:
def extract_unified_window_features_fixed(window_data, farm, sensor_mapping, 
                                    statistics=['mean', 'std', 'trend', 'max', 'min']):
    
    farm_sensors = sensor_mapping[sensor_mapping['farm'] == farm].copy()
    
    unified_features = {}
    
    for (primary, secondary), group in farm_sensors.groupby(['primary group', 'secondary group']):
        
        category_key = f"{primary}_{secondary}"
        
        mapping_sensor_names = group['sensor_name'].tolist()
        
        available_sensors = []
        for col in window_data.columns:
            root_name = extract_root_sensor_name(col)
            if root_name in mapping_sensor_names:
                available_sensors.append(col)
        
        if len(available_sensors) == 0:
            continue
        
        category_data = window_data[available_sensors]
        
        if len(available_sensors) > 1:
            category_series = category_data.mean(axis=1, skipna=True)
        else:
            category_series = category_data[available_sensors[0]]
        
        if category_series.isna().all():
            continue
        
        if 'mean' in statistics:
            unified_features[f"{category_key}_mean"] = category_series.mean()
        
        if 'std' in statistics:
            unified_features[f"{category_key}_std"] = category_series.std()
        
        if 'trend' in statistics:
            unified_features[f"{category_key}_trend"] = compute_linear_trend(category_series)
        
        if 'max' in statistics:
            unified_features[f"{category_key}_max"] = category_series.max()
        
        if 'min' in statistics:
            unified_features[f"{category_key}_min"] = category_series.min()
    
    return unified_features

In [18]:
def compute_linear_trend(series):
    
    series_clean = series.dropna()
    if len(series_clean) < 2:
        return 0.0
    
    x = np.arange(len(series_clean))
    try:
        slope, _ = np.polyfit(x, series_clean, 1)
        return slope
    except:
        return 0.0
    print('linear trend done')

In [6]:
def analyze_unified_features(dataset):
    
    feature_cols = [c for c in dataset.columns 
                   if c not in ['label', 'event_id', 'asset_id', 'farm', 'failure_type']]
    
    print(f"\nTotal unified features: {len(feature_cols)}")
    
    if len(feature_cols) == 0:
        print("ERROR: No feature columns found!")
        print("Available columns:", dataset.columns.tolist())
        return None
    
    stat_types = {}
    for stat in ['mean', 'std', 'trend', 'max', 'min']:
        count = len([c for c in feature_cols if c.endswith(f'_{stat}')])
        stat_types[stat] = count
    
    print("\nFeatures by statistic type:")
    for stat, count in stat_types.items():
        print(f"  {stat}: {count}")
    
    # Check coverage across farms
    print("\n" + "="*60)
    print("Feature Coverage by Farm:")
    print("="*60)
    
    for farm in ['A', 'B', 'C']:
        farm_data = dataset[dataset['farm'] == farm]
        if len(farm_data) == 0:
            print(f"\nFarm {farm}: NO DATA")
            continue
            
        non_null = farm_data[feature_cols].notna().sum()
        coverage = (non_null / len(farm_data) * 100).describe()
        
        print(f"\nFarm {farm} ({len(farm_data)} windows):")
        print(f"  Mean coverage: {coverage['mean']:.1f}%")
        print(f"  Min coverage: {coverage['min']:.1f}%")
        print(f"  Features with 100% coverage: {(non_null == len(farm_data)).sum()}")
    
    # Missing data analysis
    print("\n" + "="*60)
    print("Missing Data Analysis:")
    print("="*60)
    
    missing_pct = (dataset[feature_cols].isna().sum() / len(dataset) * 100).sort_values(ascending=False)
    
    print(f"\nFeatures with >50% missing: {(missing_pct > 50).sum()}")
    print(f"Features with >25% missing: {(missing_pct > 25).sum()}")
    print(f"Features with >10% missing: {(missing_pct > 10).sum()}")
    print(f"Features with 0% missing: {(missing_pct == 0).sum()}")
    
    if (missing_pct > 50).sum() > 0:
        print("\nTop 20 features with most missing data:")
        print(missing_pct.head(20))
    
    print("\n" + "="*60)
    print("Discriminative Power Analysis:")
    print("="*60)
    
    discriminative_features = []
    skipped_missing = 0
    skipped_error = 0
    
    for feature in feature_cols:
        normal_vals = dataset[dataset['label'] == 0][feature]
        failure_vals = dataset[dataset['label'] == 1][feature]
        
        # RELAXED: Skip only if >75% missing (was 50%)
        normal_missing_pct = normal_vals.isna().sum() / len(normal_vals)
        failure_missing_pct = failure_vals.isna().sum() / len(failure_vals)
        
        if normal_missing_pct > 0.75 or failure_missing_pct > 0.75:
            skipped_missing += 1
            continue
        
        normal_clean = normal_vals.dropna()
        failure_clean = failure_vals.dropna()
        
        if len(normal_clean) < 3 or len(failure_clean) < 3:
            skipped_error += 1
            continue
        
        try:
            mean_diff = failure_clean.mean() - normal_clean.mean()
            pooled_std = np.sqrt((normal_clean.std()**2 + failure_clean.std()**2) / 2)
            cohens_d = abs(mean_diff / pooled_std) if pooled_std > 0 else 0
            
            discriminative_features.append({
                'feature': feature,
                'cohens_d': cohens_d,
                'mean_diff': mean_diff,
                'normal_missing_pct': normal_missing_pct,
                'failure_missing_pct': failure_missing_pct
            })
        except Exception as e:
            skipped_error += 1
            continue
    
    print(f"\nFeatures analyzed: {len(discriminative_features)}")
    print(f"Skipped (>75% missing): {skipped_missing}")
    print(f"Skipped (errors/insufficient data): {skipped_error}")
    
    if len(discriminative_features) == 0:
        print("\nERROR: No features could be analyzed!")
        print("This suggests all features have too much missing data.")
        print("\nDEBUG: Check first few feature columns:")
        for col in feature_cols[:5]:
            print(f"  {col}: {dataset[col].isna().sum()}/{len(dataset)} missing")
        return None
    
    df_disc = pd.DataFrame(discriminative_features).sort_values('cohens_d', ascending=False)
    
    print(f"Features with d > 1.0: {(df_disc['cohens_d'] > 1.0).sum()}")
    print(f"Features with d > 0.8: {(df_disc['cohens_d'] > 0.8).sum()}")
    print(f"Features with d > 0.6: {(df_disc['cohens_d'] > 0.6).sum()}")
    print(f"Features with d > 0.5: {(df_disc['cohens_d'] > 0.5).sum()}")
    
    print("\nTop 20 discriminative unified features:")
    print(df_disc.head(20)[['feature', 'cohens_d', 'mean_diff']].to_string(index=False))
    
    return df_disc


In [7]:
def select_unified_features(dataset, cohens_d_threshold=0.6, max_features=20, 
                           correlation_threshold=0.9):
    
    feature_cols = [c for c in dataset.columns 
                   if c not in ['label', 'event_id', 'asset_id', 'farm', 'failure_type']]
    
    print(f"\nTotal feature columns: {len(feature_cols)}")
    
    feature_analysis = []
    skipped_missing = 0
    skipped_error = 0
    
    for feature in feature_cols:
        normal_vals = dataset[dataset['label'] == 0][feature]
        failure_vals = dataset[dataset['label'] == 1][feature]
        
        # RELAXED: Skip only if >75% missing (was 50%)
        normal_missing_pct = normal_vals.isna().sum() / len(normal_vals)
        failure_missing_pct = failure_vals.isna().sum() / len(failure_vals)
        
        if normal_missing_pct > 0.75 or failure_missing_pct > 0.75:
            skipped_missing += 1
            continue
        
        normal_clean = normal_vals.dropna()
        failure_clean = failure_vals.dropna()
        
        if len(normal_clean) < 3 or len(failure_clean) < 3:
            skipped_error += 1
            continue
        
        try:
            mean_diff = failure_clean.mean() - normal_clean.mean()
            pooled_std = np.sqrt((normal_clean.std()**2 + failure_clean.std()**2) / 2)
            cohens_d = abs(mean_diff / pooled_std) if pooled_std > 0 else 0
            
            feature_analysis.append({
                'feature': feature,
                'cohens_d': cohens_d
            })
        except Exception as e:
            skipped_error += 1
            continue
    
    print(f"\nFeatures analyzed: {len(feature_analysis)}")
    print(f"Skipped (>75% missing): {skipped_missing}")
    print(f"Skipped (errors/insufficient data): {skipped_error}")
    
    if len(feature_analysis) == 0:
        print("\nERROR: No features could be analyzed!")
        return []
    
    df_analysis = pd.DataFrame(feature_analysis).sort_values('cohens_d', ascending=False)
    
    selected_features = []
    candidates = df_analysis[df_analysis['cohens_d'] > cohens_d_threshold]['feature'].tolist()
    
    print(f"\nCandidates (d > {cohens_d_threshold}): {len(candidates)}")
    
    if len(candidates) == 0:
        print(f"\nWARNING: No features exceed Cohen's d threshold of {cohens_d_threshold}")
        print("Lowering threshold to include top 10 features...")
        candidates = df_analysis.head(10)['feature'].tolist()
    
    for candidate in candidates:
        if len(selected_features) >= max_features:
            break
        
        if len(selected_features) == 0:
            selected_features.append(candidate)
            continue
        
        # Check correlation with imputation for missing values
        try:
            subset = dataset[selected_features + [candidate]].copy()
            
            # Impute missing values
            from sklearn.impute import SimpleImputer
            imputer = SimpleImputer(strategy='median')
            subset_imputed = pd.DataFrame(
                imputer.fit_transform(subset),
                columns=subset.columns
            )
            
            corr_check = subset_imputed.corr()
            max_corr = corr_check[candidate][:-1].abs().max()
            
            if max_corr < correlation_threshold:
                selected_features.append(candidate)
        except Exception as e:
            continue
    
    print(f"\nSelected {len(selected_features)} features")
    print(f"(max_corr < {correlation_threshold}, d > {cohens_d_threshold})")
    
    if len(selected_features) > 0:
        print("\n\nFINAL UNIFIED FEATURE SET:")
        print("="*60)
        for f in selected_features:
            d = df_analysis[df_analysis['feature']==f]['cohens_d'].iloc[0]
            print(f"  {f:<50} d={d:.3f}")
    else:
        print("\nERROR: No features selected!")
    
    return selected_features

In [8]:
def evaluate_unified_model(dataset, selected_features):

    print("\n" + "="*80)
    print("UNIFIED MODEL EVALUATION")
    print("="*80)
    
    X = dataset[selected_features]
    y = dataset['label']
    
    print(f"\nFeatures: {len(selected_features)}")
    print(f"Total samples: {len(X)}")
    print(f"Failures: {y.sum()}")
    print(f"Normal: {(y==0).sum()}")
    
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(
        imputer.fit_transform(X),
        columns=X.columns,
        index=X.index
    )
    
    print(f"\nMissing values before imputation: {X.isna().sum().sum()}")
    print(f"Missing values after imputation: {X_imputed.isna().sum().sum()}")
    
    loo = LeaveOneOut()
    y_true = []
    y_pred = []
    y_pred_proba = []
    
    print("\nRunning Leave-One-Out Cross-Validation...")
    
    for train_idx, test_idx in loo.split(X_imputed):
        X_train, X_test = X_imputed.iloc[train_idx], X_imputed.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        clf = RandomForestClassifier(
            n_estimators=100,
            max_depth=5,
            random_state=42,
            class_weight='balanced'
        )
        clf.fit(X_train, y_train)
        
        y_true.append(y_test.values[0])
        y_pred.append(clf.predict(X_test)[0])
        y_pred_proba.append(clf.predict_proba(X_test)[0, 1])
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred_proba = np.array(y_pred_proba)
    
    print("\n" + "="*60)
    print("OVERALL PERFORMANCE")
    print("="*60)
    
    cm = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(cm)
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Normal', 'Failure']))
    
    failure_indices = np.where(y_true == 1)[0]
    detected = y_pred[failure_indices].sum()
    total = len(failure_indices)
    
    print(f"\n{'='*60}")
    print(f"FAILURES DETECTED: {detected} / {total} ({detected/total*100:.1f}%)")
    print(f"{'='*60}")
    
    print("\n" + "="*60)
    print("PERFORMANCE BY FARM")
    print("="*60)
    
    for farm in ['A', 'B', 'C']:
        farm_mask = dataset['farm'] == farm
        farm_indices = np.where(farm_mask)[0]
        
        y_true_farm = y_true[farm_indices]
        y_pred_farm = y_pred[farm_indices]
        
        failure_mask = y_true_farm == 1
        if failure_mask.sum() == 0:
            continue
        
        farm_detected = y_pred_farm[failure_mask].sum()
        farm_total = failure_mask.sum()
        farm_recall = farm_detected / farm_total if farm_total > 0 else 0
        
        normal_mask = y_true_farm == 0
        farm_fp = (y_pred_farm[normal_mask] == 1).sum()
        farm_tn = (y_pred_farm[normal_mask] == 0).sum()
        farm_precision = farm_detected / (farm_detected + farm_fp) if (farm_detected + farm_fp) > 0 else 0
        
        print(f"\nFarm {farm}:")
        print(f"  Total failures: {farm_total}")
        print(f"  Detected: {farm_detected} ({farm_recall:.0%})")
        print(f"  False alarms: {farm_fp}/{farm_tn + farm_fp}")
        print(f"  Precision: {farm_precision:.0%}")
    
    return {
        'y_true': y_true,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'confusion_matrix': cm
    }

In [16]:
def extract_root_sensor_name(col_name):
    
    suffixes = ['_min', '_max', '_avg', '_std', '_std_dev']
    
    for suffix in suffixes:
        if col_name.endswith(suffix):
            return col_name[:-len(suffix)]
    
    # If no suffix found, return as-is
    return col_name


def extract_unified_window_features_fixed(window_data, farm, sensor_mapping, 
                                          statistics=['mean', 'std', 'trend']):

    farm_sensors = sensor_mapping[sensor_mapping['farm'] == farm].copy()
    
    unified_features = {}
    
    for (primary, secondary), group in farm_sensors.groupby(['primary group', 'secondary group']):
        
        category_key = f"{primary}_{secondary}"
        
        mapping_sensor_names = group['sensor_name'].tolist()
        
        available_sensors = []
        for col in window_data.columns:
            root_name = extract_root_sensor_name(col)
            if root_name in mapping_sensor_names:
                available_sensors.append(col)
        
        if len(available_sensors) == 0:
            continue
        
        category_data = window_data[available_sensors]
        
        if len(available_sensors) > 1:
            category_series = category_data.mean(axis=1, skipna=True)
        else:
            category_series = category_data[available_sensors[0]]
        
        if category_series.isna().all():
            continue
        
        if 'mean' in statistics:
            unified_features[f"{category_key}_mean"] = category_series.mean()
        
        if 'std' in statistics:
            unified_features[f"{category_key}_std"] = category_series.std()
        
        if 'trend' in statistics:
            unified_features[f"{category_key}_trend"] = compute_linear_trend(category_series)
    
    return unified_features


In [21]:
sensor_mapping_v2 = pd.read_csv(BASE_DIR+'\\sensor_mapping_v2.csv')

scada_dict = {
    'A': get_farm_scada_chunked(farm='A'),
    'B': get_farm_scada_chunked(farm='B'),
    'C': get_farm_scada_chunked(farm='C')
}

event_info_dict = {
    'A': get_event_info(farm='A'),
    'B': get_event_info(farm='B'),
    'C': get_event_info(farm='C')
}

power_sensors = {
    'A': 'power_30_avg',
    'B': 'power_62_avg', 
    'C': 'power_2_avg'
}
print("\nRebuilding unified dataset with merged categories...")
print("="*80)

# Rebuild dataset using the merged sensor mapping
dataset_unified_seeded = build_unified_dataset(
    scada_dict=scada_dict,
    event_info_dict=event_info_dict,
    sensor_mapping=sensor_mapping_v2,
    power_sensors=power_sensors,
    power_threshold=0.1,
    window_hours=24,
    buffer_days=30,
    random_seed=42  # Add this parameter!
)

# Quick diagnostic
print(f"\nDataset shape: {dataset_unified_seeded.shape}")
print(f"Total windows: {len(dataset_unified_seeded)}")
print(f"Failure windows: {(dataset_unified_seeded['label']==1).sum()}")
print(f"Normal windows: {(dataset_unified_seeded['label']==0).sum()}")

# Check missing data
feature_cols = [c for c in dataset_unified_seeded.columns 
               if c not in ['label', 'event_id', 'asset_id', 'farm', 'failure_type']]
missing_pct = (dataset_unified_seeded[feature_cols].isna().sum() / len(dataset_unified_seeded) * 100)

print(f"\nFeatures: {len(feature_cols)}")
print(f"Missing data: {missing_pct.mean():.1f}% average")
print(f"Features with <25% missing: {(missing_pct < 25).sum()}")
print(f"Features with <50% missing: {(missing_pct < 50).sum()}")

# Analyze the unified features
df_discriminative_v2 = analyze_unified_features(dataset_unified_seeded)

# Select features
selected_unified_features = select_unified_features(
    dataset=dataset_unified_seeded,
    cohens_d_threshold=0.5,  # Relaxed from 0.6
    max_features=20,
    correlation_threshold=0.9
)

# Evaluate model
results_unified_v2 = evaluate_unified_model(
    dataset=dataset_unified_seeded, 
    selected_features=selected_unified_features
)


Loading pre-processed Farm A...
Loading pre-processed Farm B...
Loading pre-processed Farm C...

Rebuilding unified dataset with merged categories...

Processing Farm A...
  Failure windows: 12
  Normal windows: 65

Processing Farm B...
  Failure windows: 6
  Normal windows: 180

Processing Farm C...
  Failure windows: 27
  Normal windows: 424

UNIFIED DATASET CREATED
Total windows: 714
Failure windows: 45
Normal windows: 669
Features: 78

Windows per farm:
farm
C    451
B    186
A     77
Name: count, dtype: int64

Dataset shape: (714, 83)
Total windows: 714
Failure windows: 45
Normal windows: 669

Features: 78
Missing data: 23.9% average
Features with <25% missing: 42
Features with <50% missing: 66

Total unified features: 78

Features by statistic type:
  mean: 26
  std: 26
  trend: 26
  max: 0
  min: 0

Feature Coverage by Farm:

Farm A (77 windows):
  Mean coverage: 61.5%
  Min coverage: 0.0%
  Features with 100% coverage: 48

Farm B (186 windows):
  Mean coverage: 61.5%
  Min cove